<a href="https://colab.research.google.com/github/daewpark/web_scrapping/blob/main/eunpyung_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Package 설치

In [1]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install smtplib

     |████████████████████████████████| 911kB 13.5MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86

### Pactkage 호출 및 로드

In [2]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920x1080')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [3]:
import urllib.request
from bs4 import BeautifulSoup
import re

### Selenium WebDriver 구동 및 검색결과 Parsing

In [4]:
url = "https://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001180"
driver.get(url)
time.sleep(1)

In [5]:
# 로드 된 웹페이지에서 검색창의 Name 을 찾아서 검색어 입력
search = driver.find_element_by_name("searTot")
search.send_keys("원아모집") # headless 모드에서 chrome_options.add_argument('--window-size=1920x1080') 옵션이 없으면 에러 발생하므로 옵션을 반드시 넣을 것
search.submit()


In [6]:
# 검색결과 파싱
driver.switch_to.window(driver.window_handles[-1]) #검색결과가 다른 탭에서 열리므로 새롭게 열린 탭으로 이동
search_html = driver.page_source
soup = BeautifulSoup(search_html,'html.parser')
title = soup.find_all(class_='btnS_nw')

### Pandas DataFrame 으로 결과저장

In [7]:
import pandas as pd

result_list = []

for i in title :
    ititle = i.attrs['title']
    ilink = i.attrs['href']
    result_list.append([ititle,ilink])

df = pd.DataFrame(result_list, columns = ["title","link"])
print(df)

                      title                                               link
0   구립 물빛캐슬어린이집 원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
1  『구립 응암행복어린이집』원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
2  『구립 응암샛별어린이집』원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
3    구립 이든어린이집 원아 모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...


### 스크래핑 결과 메일로 보내기

In [ ]:
import smtplib;
# from email.mime.base import MIMEBase;
# 메일 메시지를 만드는 모듈 (MIMEBase는 이하 MIMEMultipart, MIMEText, MIMEApplication, MIMEImage, MIMEAudio)의 상위 모듈이다.
# 굳이 선언할 필요없다.

from email.mime.multipart import MIMEMultipart                                  # 메일의 Data 영역의 메시지를 만드는 모듈 (MIMEText, MIMEApplication, MIMEImage, MIMEAudio가 attach되면 바운더리 형식으로 변환)
from email.mime.text import MIMEText                                            # 메일의 본문 내용을 만드는 모듈
from email.mime.application import MIMEApplication                              # 메일의 첨부 파일을 base64 형식으로 변환
from email.mime.image import MIMEImage                                          # 메일의 이미지 파일을 base64 형식으로 변환(Content-ID 생성)c
from email.mime.audio import MIMEAudio                                          # 메일의 음악 파일을 base64 형식으로 변환(Content-ID 생성)
import io # 파일 IO

2021-02-25


In [ ]:
#메일 발송시간을 알리기 위하여 timestamp 라이브러리 호출
import datetime as dt
today = dt.date.today()
today_string = today.strftime("%Y-%m-%d")

# 메일 서버와 통신하기 전에 메시지를 만든다.
data = MIMEMultipart()
data['From'] = "dwn.park@gmail.com"
data['To'] = "dwn.park@gmail.com"
#data['Cc'] = "nowonbun@gmail.com"
#data['Bcc'] = "nowonbun@gmail.com"
data['Subject'] = "은평구청 원아모집 공고 알림봇" + today_string
print (data['Subject'])

In [ ]:
# 첨부 파일 추가
with open("test.xlsx", "rb") as fp :
    part = MIMEApplication(fp.read(), Name = "test.xlsx") # Name은 메일 수신자에서 설정되는 파일 이름
    data.attach (part) # Data 영역의 메시지에 바운더리 추가

# 이미지 파일 추가
with open("capture.png", 'rb') as fp:

# Name은 메일 수신자에서 설정되는 파일 이름
img = MIMEImage(fp.read(), Name = "capture.png")

# 해더에 Content-ID 추가(본문 내용에서 cid로 링크를 걸 수 있다.)
img.add_header('Content-ID', '<capture>')

# Data 영역의 메시지에 바운더리 추가
data.attach(img)

# 텍스트 형식의 본문 내용
#msg = MIMEText("Hello world", 'plain');

# Html 형식의 본문 내용 (cid로 이미 첨부 파일을 링크했다.)
msg = MIMEText("Hello Test<br /><img src='cid:capture'>", 'html')

# Data 영역의 메시지에 바운더리 추가
data.attach(msg);

# 메시지를 확인한다.
print(data);


In [ ]:

server = smtplib.SMTP_SSL('smtp.gmail.com',465); # 메일 서버와 telnet 통신 개시
#server = smtplib.SMTP('smtp.gmail.com',587);
# 메일 통신시 디버그
server.set_debuglevel(1);
# 헤로 한번 해주자.(의미 없음)
server.ehlo();
# tls 설정 주문 - tls 587 포트의 경우
#server.starttls();
# 헤로 또 해주자.(의미 없음)
server.ehlo();
# 로그인 한다.
server.login("id", "password");
# 심심하니 또 헤로 해주자.(의미 없음)
server.ehlo();
# MAIL(송신자) 설정
sender = data['From'];
# RCPT(수신자), 리스트로 보낸다.
# 수신자 추가
receiver = data['To'].split(",");
# 참조자 추가
if data['Cc'] is not None:
receiver += data['Cc'].split(",");
# 숨은 참조자 추가
if data['Bcc'] is not None:
receiver += data['Bcc'].split(",");
# 메일 프로토콜 상 MAIL, RCPT, DATA 순으로 메시지를 보내야 하는데 이걸 sendmail함수에서 자동으로 해준다.
server.sendmail(sender, receiver, data.as_string());
# QUIT을 보내고 접속을 종료하고 메일을 보낸다.
server.quit();
